# **Importing The Libraries**

In [ ]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# **Loading The Data for Modelling**

In [ ]:
import pickle

# 1. Load the processed arrays
print("Loading data arrays...")
data = np.load('processed_data.npz')
X_train, y_train = data['X_train'], data['y_train']
X_val, y_val = data['X_val'], data['y_val']
X_test, y_test = data['X_test'], data['y_test']

# 2. Load metadata
print("Loading metadata...")
with open('metadata.pkl', 'rb') as f:
    metadata = pickle.load(f)

label_lookup = metadata['label_lookup']
reverse_lookup = metadata['reverse_lookup']
num_classes = metadata['num_classes']
IMG_SIZE = metadata['IMG_SIZE']

# 3. Re-initialize the Data Augmentation (Generators)
# Note: We define the generator logic here based on the data loaded
train_datagen = ImageDataGenerator(
    rotation_range=25,
    zoom_range=0.2,
    width_shift_range=0.20,
    height_shift_range=0.20,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator() # No augmentation for validation

# 4. Create the flow from the loaded data
train_gen = train_datagen.flow(X_train, y_train, batch_size=64, shuffle=True)
val_gen = val_datagen.flow(X_val, y_val, batch_size=64)

print(f"✅ Setup Complete.")
print(f"Training Samples: {X_train.shape[0]}")
print(f"Validation Samples: {X_val.shape[0]}")
print(f"Number of Classes: {num_classes}")
print("You can now proceed to build_cnn()")

Loading data arrays...


FileNotFoundError: [Errno 2] No such file or directory: 'processed_data.npz'

# **Modelling**

In [ ]:
def build_cnn(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.25),

        layers.Conv2D(64, (3,3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.30),

        layers.Conv2D(128, (3,3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.40),

        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.50),

        layers.Dense(num_classes, activation='softmax')
    ])

    optimizer = Adam(learning_rate=0.0008)

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [ ]:
model = build_cnn((IMG_SIZE, IMG_SIZE, 1), num_classes)
model.summary()

In [ ]:
callbacks = [
    EarlyStopping(monitor="val_loss", patience=7, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.3, patience=3, min_lr=1e-6)
]

In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    callbacks=callbacks,
    verbose=1
)

**Model Performance**

In [ ]:
history_df = pd.DataFrame(history.history)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_df["accuracy"], label="Train Acc")
plt.plot(history_df["val_accuracy"], label="Val Acc")
plt.title("Accuracy")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_df["loss"], label="Train Loss")
plt.plot(history_df["val_loss"], label="Val Loss")
plt.title("Loss")
plt.legend()
plt.show()

**Test Accuracy**

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print("Test Accuracy:", round(test_acc * 100, 2), "%")
print("Test Loss:", test_loss)

In [ ]:
y_pred = model.predict(X_test)
y_pred_int = np.argmax(y_pred, axis=1)
y_true_int = np.argmax(y_test, axis=1)

target_names = [reverse_lookup[i] for i in range(num_classes)]

print(classification_report(y_true_int, y_pred_int, target_names=target_names))

**Confusion Matrix**

In [ ]:
cm = confusion_matrix(y_true_int, y_pred_int)
plt.figure(figsize=(10, 8))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=target_names,
    yticklabels=target_names
)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Save the model to a single file
# .keras is the modern, zipped format for TensorFlow 2.x
model.save('hand_gesture_model.keras')

print("Model saved successfully as 'hand_gesture_model.keras'")